In [ ]:
%pip install wildlife-datasets

In [ ]:
%pip install wildlife-tools

In [ ]:
%pip install timm

## **Testing if able to download datasets**

In [ ]:
#Import wildlife-datasets Library
from wildlife_datasets import datasets, splits

#Download dataset
datasets.ATRW.get_data('data/ATRW')

#Load metadata
metadata = datasets.ATRW('data/ATRW')

#Get 80/20 training/test split
splitter = splits.ClosedSetSplit(0.8)
splitter.split(metadata.df)


# **SIFT**

# **Acrface_Loss accuracy**

In [ ]:
import itertools
from itertools import chain

import timm
import pandas as pd
import torchvision.transforms as T
from torch.optim import SGD

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.train import ArcFaceLoss, BasicTrainer

import timm
import numpy as np
from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset
import torchvision.transforms as T
from wildlife_datasets import datasets, splits

from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

### Training IPanda50

TRIPLET_LOSS

**NEW DATASET  DogFaceNet**

Arcface